In [1]:
# Install necessary packages
import langchain
import openai
import pinecone
import pandas as pd
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI   
from pinecone import Pinecone, ServerlessSpec
import os





c:\Users\ramre\OneDrive\Desktop\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## lets read the documents
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents
documents = read_doc(r"C:\Users\ramre\Downloads/")
documents





[Document(metadata={'producer': 'iOS Version 18.5 (Build 22F76) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20250625215016Z00'00'", 'moddate': "D:20250625215016Z00'00'", 'source': 'C:\\Users\\ramre\\Downloads\\driver’s license_Ram Reddy.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'iOS Version 18.5 (Build 22F76) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20250625215016Z00'00'", 'moddate': "D:20250625215016Z00'00'", 'source': 'C:\\Users\\ramre\\Downloads\\driver’s license_Ram Reddy.pdf', 'total_pages': 2, 'page': 1, 'page_label': '2'}, page_content=''),
 Document(metadata={'producer': 'macOS Version 15.5 (Build 24F74) Quartz PDFContext, AppendMode 1.1', 'creator': 'PyPDF', 'creationdate': "D:20250625215540Z00'00'", 'moddate': "D:20250702193552Z00'00'", 'source': 'C:\\Users\\ramre\\Downloads\\Employment authorization_card.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content=''),
 Doc

In [3]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Step 1: Load all PDFs from a folder
def read_doc(directory):
    loader = PyPDFDirectoryLoader(directory)
    documents = loader.load()
    return documents

raw_documents = read_doc(r"C:\Users\ramre\Downloads")
print(f"✅ Loaded {len(raw_documents)} raw pages")

# Step 2: Split into chunks
def doc_to_chunks(documents, chunk_size=1000, chunk_overlap=100):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_documents(documents)

documents = doc_to_chunks(raw_documents)
print(f"✅ Split into {len(documents)} chunks")


✅ Loaded 38 raw pages
✅ Split into 139 chunks


In [4]:
documents = doc_to_chunks(documents)
documents

[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with RenderCV', 'creationdate': '2025-09-23T20:22:03+00:00', 'author': 'Ram Reddy', 'keywords': '', 'moddate': '2025-09-23T20:22:03+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': 'Ram Reddy CV', 'trapped': '/False', 'source': 'C:\\Users\\ramre\\Downloads\\Ram Baddam Resume.pdf', 'total_pages': 3, 'page': 0, 'page_label': ''}, page_content='Ram Reddy Baddam\nAustin, TX | ram.baddam4377@gmail.com | +1512-537-5632 | www.linkedin.com/in/ramreddybaddam\nSummary\nGenerative AI & Machine Learning Engineer with 5 years of experience designing, developing, and deploying\nproduction-grade AI/ML systems across finance, healthcare, and enterprise domains. Proven expertise in building\nLLM-powered applications using LangChain, OpenAI, Hugging Face, and Retrieval-Augmented Generation (RAG)\npipelines, as well as fine-tuning transformer mode

In [5]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_5jREiS_TsSj9oFWfT6aLCRx8BWN5DWvejRLP6NqfZoaLhSasnHDr53D3GEvF4U8XCyLHpw")


In [6]:
index_name = "bge-vectors"

# Create index only if it doesn’t exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,              # BGE-large = 1024-dim
        metric="cosine",             # good for similarity search
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"       # must match your account region
        )
    )
    print(f"✅ Created index: {index_name}")
else:
    print(f"ℹ️ Index '{index_name}' already exists")

# Connect to index
index = pc.Index(index_name)
print("✅ Connected to Pinecone index")


ℹ️ Index 'bge-vectors' already exists
✅ Connected to Pinecone index


In [7]:
from sentence_transformers import SentenceTransformer

# Load BGE model
model = SentenceTransformer("BAAI/bge-large-en-v1.5")

# Extract texts from your chunked documents
texts = [doc.page_content for doc in documents]

# Generate embeddings
embeddings = model.encode(texts, show_progress_bar=True)
print(f"Created {len(embeddings)} embeddings of size {embeddings.shape[1]}")


Batches: 100%|██████████| 5/5 [01:49<00:00, 21.92s/it]

Created 139 embeddings of size 1024


In [8]:
# Build vectors for Pinecone
vectors = [
    {
        "id": str(i),
        "values": embeddings[i].tolist(),
        "metadata": {
            "source": documents[i].metadata.get("source", ""),
            "page": documents[i].metadata.get("page", 0),
            "text": texts[i][:200]  # snippet for context
        }
    }
    for i in range(len(texts))
]

# Upload to Pinecone
index.upsert(vectors=vectors)
print(f"Uploaded {len(vectors)} chunks into Pinecone")


Uploaded 139 chunks into Pinecone


In [9]:
query = "What projects has Ram Reddy Baddam worked on?"
query_emb = model.encode([query]).tolist()

results = index.query(vector=query_emb[0], top_k=5, include_metadata=True)

for match in results["matches"]:
    print(f"Score: {match['score']:.3f}")
    print(f"Text: {match['metadata']['text']}\n")


Score: 0.694
Text: Ram Reddy Baddam
Austin, TX | ram.baddam4377@gmail.com | +1512-537-5632 | www.linkedin.com/in/ramreddybaddam
Summary
Generative AI & Machine Learning Engineer with 5 years of experience designing, dev

Score: 0.694
Text: Ram Reddy Baddam
Austin, TX | ram.baddam4377@gmail.com | +1512-537-5632 | www.linkedin.com/in/ramreddybaddam
Summary
Generative AI & Machine Learning Engineer with 5 years of experience designing, dev

Score: 0.694
Text: Ram Reddy Baddam
Austin, TX | ram.baddam4377@gmail.com | +1512-537-5632 | www.linkedin.com/in/ramreddybaddam
Summary
Generative AI & Machine Learning Engineer with 5 years of experience designing, dev

Score: 0.649
Text: Ram Reddy Baddam
Austin, TX| +1512-537-5632 | ram.baddam4377@gmail.com | LinkedIn
Summary
Data Engineer with expertise in Python and SQL, experienced in deploying scalable data pipelines and models fo

Score: 0.648
Text: Ram Reddy Baddam
Austin, TX| +1512-537-5632 | ram.baddam4377@gmail.com | LinkedIn
Summary
Softwar

In [10]:
# ## cosine similarity search from vector db.
# def retrive_query(query, top_k=3):
#     match = index.similarity_search(query, top_k=top_k)
#     return match

In [11]:
from langchain_community.llms import Ollama
from langchain.chains.question_answering import load_qa_chain
from langchain.schema import Document

# 1. Initialize Ollama LLM
llm = Ollama(model="mistral")  # you can also use "llama2" if pulled

# 2. Build a QA chain
chain = load_qa_chain(llm, chain_type="stuff")

# 3. Ask a query (Pinecone must already return results)
query = "how many years of experiance does ram reddy have?"

# Convert Pinecone matches into LangChain Document objects
docs = [
    Document(page_content=match['metadata']['text'], metadata=match['metadata'])
    for match in results["matches"]
]

# 4. Run QA with invoke (preferred over run)
response = chain.invoke({"input_documents": docs, "question": query})

print("🔎 Query:", query)
print("💡 Answer:", response["output_text"])


C:\Users\ramre\AppData\Local\Temp\ipykernel_7636\3202864349.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")  # you can also use "llama2" if pulled
C:\Users\ramre\AppData\Local\Temp\ipykernel_7636\3202864349.py:9: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chain

🔎 Query: how many years of experiance does ram reddy have?
💡 Answer:  Ram Reddy Baddam has 5 years of experience according to the context provided.


In [12]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")  # or "llama2"
llm

Ollama()

In [13]:
# desoign prompt tempelate
from langchain_core.prompts import ChatPromptTemplate   
prompt=ChatPromptTemplate.from_template("""You are a helpful AI assistant. Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.<context>
                                         {context}
Question: {input}""") 


In [14]:
# create a long chain with prompt and llm
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm, prompt)